## Downloading the File

In [8]:
import boto3

s3_client = boto3.client('s3')

# Specify the S3 bucket name and file details
bucket_name = 'savingbuckett5'
s3_file_key = 'Video-Processing/sample.mp4'
local_file_name = 'sample.mp4'

try:
    s3_client.download_file(bucket_name, s3_file_key, local_file_name)
    print(f"File '{s3_file_key}' downloaded successfully from '{bucket_name}' and saved as '{local_file_name}'.")
except Exception as e:
    print(f"Error downloading file from S3: {str(e)}")


KeyboardInterrupt: 

## Uploading the File

In [4]:
import boto3
from botocore.exceptions import NoCredentialsError, ClientError

# Initialize the S3 client
s3_client = boto3.client('s3')

bucket_name = 'savingbuckett5'
s3_file_key = 'Video-Processing/sample.mp4'
local_file_name = 'sample.mp4'

try:
    # Upload the file
    s3_client.upload_file(local_file_name, bucket_name, s3_file_key)
    print(f"File '{local_file_name}' uploaded successfully to '{bucket_name}' as '{s3_file_key}'.")
except FileNotFoundError:
    print(f"The file '{local_file_name}' was not found.")
except NoCredentialsError:
    print("Credentials not available.")
except ClientError as e:
    print(f"Error uploading file: {str(e)}")


The file 'sample.mp4' was not found.


## Getting the information of the File

In [7]:
import boto3
import cv2
import os
import tempfile

# Initialize the S3 client
s3_client = boto3.client('s3')

# Specify the S3 bucket and video file key
bucket_name = 'savingbuckett5'
s3_file_key = 'Video-Processing/sample.mp4'

try:
    # Create a temporary file to save the video from S3
    with tempfile.NamedTemporaryFile(suffix=".mp4", delete=False) as temp_video_file:
        temp_video_path = temp_video_file.name
        
        # Download the video file from S3 and save to the temporary file
        s3_client.download_file(bucket_name, s3_file_key, temp_video_path)
        print(f"File '{s3_file_key}' downloaded successfully from '{bucket_name}' and saved as '{temp_video_path}'.")

    # Read and display the video using OpenCV
    video_capture = cv2.VideoCapture(temp_video_path)

    if not video_capture.isOpened():
        print("Error opening video stream or file")
        exit()

    # Display video frame by frame
    while video_capture.isOpened():
        ret, frame = video_capture.read()
        
        if ret:
            # Display the current frame
            cv2.imshow('Video', frame)

            # Press 'q' on the keyboard to exit
            if cv2.waitKey(25) & 0xFF == ord('q'):
                break
        else:
            break

    # Release the video capture object
    video_capture.release()
    cv2.destroyAllWindows()

    # Clean up: Remove the temporary file
    os.remove(temp_video_path)
    print(f"Temporary file '{temp_video_path}' deleted.")

except Exception as e:
    print(f"Error reading video from S3: {str(e)}")


File 'Video-Processing/sample.mp4' downloaded successfully from 'savingbuckett5' and saved as '/var/folders/c1/9m9gfrj553l8956nyk7q8gw40000gn/T/tmpe1ylbaru.mp4'.


KeyboardInterrupt: 

## Storing Text Information

In [11]:
import pymysql

# Database connection details
RDS_HOST = "database-2.cnqamusmkwon.eu-north-1.rds.amazonaws.com"
RDS_PORT = 3306  # MySQL default port
DB_USER = "root"
DB_PASSWORD = "mkmk162345"
DB_NAME = "traffic"


# Function to connect to the database
def get_connection():
    return pymysql.connect(
        host=RDS_HOST,
        port=RDS_PORT,
        user=DB_USER,
        password=DB_PASSWORD,
        database=DB_NAME,
        cursorclass=pymysql.cursors.DictCursor
    )

# Function to increment road_in, calculate road_current, and update the table
def increment_road_in(id_value, increment_value):
    try:
        connection = get_connection()
        with connection.cursor() as cursor:
            # Step 1: Retrieve the current data
            select_sql = "SELECT id, road_in, road_out, road_current FROM traffic_counter_road WHERE id = %s"
            cursor.execute(select_sql, (id_value,))
            result = cursor.fetchone()
        
        if result:
            # Step 2: Modify road_in and calculate road_current
            new_road_in = result['road_in'] + increment_value
            new_road_current = new_road_in - result['road_out']
            
            # Step 3: Update the table
            with connection.cursor() as cursor:
                update_sql = """
                UPDATE traffic_counter_road
                SET road_in = %s, road_current = %s
                WHERE id = %s
                """
                cursor.execute(update_sql, (new_road_in, new_road_current, id_value))
            
            # Commit the changes
            connection.commit()
            print(f"Updated road_in to {new_road_in} and road_current to {new_road_current} for id {id_value}")
        
    except pymysql.MySQLError as e:
        print(f"Error: {e}")
    
    finally:
        if connection:
            connection.close()

# Function to increment road_out, calculate road_current, and update the table
def increment_road_out(id_value, increment_value):
    try:
        connection = get_connection()
        with connection.cursor() as cursor:
            # Step 1: Retrieve the current data
            select_sql = "SELECT id, road_in, road_out, road_current FROM traffic_counter_road WHERE id = %s"
            cursor.execute(select_sql, (id_value,))
            result = cursor.fetchone()
        
        if result:
            # Step 2: Modify road_out and calculate road_current
            new_road_out = result['road_out'] + increment_value
            new_road_current = result['road_in'] - new_road_out
            
            # Step 3: Update the table
            with connection.cursor() as cursor:
                update_sql = """
                UPDATE traffic_counter_road
                SET road_out = %s, road_current = %s
                WHERE id = %s
                """
                cursor.execute(update_sql, (new_road_out, new_road_current, id_value))
            
            # Commit the changes
            connection.commit()
            print(f"Updated road_out to {new_road_out} and road_current to {new_road_current} for id {id_value}")
        
    except pymysql.MySQLError as e:
        print(f"Error: {e}")
    
    finally:
        if connection:
            connection.close()

# # Example usage:
increment_road_in(1, 1) 
# increment_road_out(1, 1)


Updated road_in to 224 and road_current to 196 for id 1


In [13]:
# RDS Details
RDS_HOST = "database-2.cnqamusmkwon.eu-north-1.rds.amazonaws.com"
RDS_PORT = 3306 
DB_USER = "root"
DB_PASSWORD = "mkmk162345"
DB_NAME = "traffic"

def get_connection():
    return pymysql.connect(
        host=RDS_HOST,
        port=RDS_PORT,
        user=DB_USER,
        password=DB_PASSWORD,
        database=DB_NAME,
        cursorclass=pymysql.cursors.DictCursor
    )

def increment_road(id_value, increment_value, is_in=True):
    try:
        connection = get_connection()
        with connection.cursor() as cursor:
            select_sql = "SELECT id, road_in, road_out, road_current FROM traffic_counter_road WHERE id = %s"
            cursor.execute(select_sql, (id_value,))
            result = cursor.fetchone()
        
        if result:
            with connection.cursor() as cursor:
                if is_in:
                    new_road_in = result['road_in'] + increment_value
                    new_road_current = new_road_in - result['road_out']
                    update_sql = """
                    UPDATE traffic_counter_road
                    SET road_in = %s, road_current = %s
                    WHERE id = %s
                    """
                    cursor.execute(update_sql, (new_road_in, new_road_current, id_value))
                else:
                    new_road_out = result['road_out'] + increment_value
                    new_road_current = result['road_in'] - new_road_out
                    update_sql = """
                    UPDATE traffic_counter_road
                    SET road_out = %s, road_current = %s
                    WHERE id = %s
                    """
                    cursor.execute(update_sql, (new_road_out, new_road_current, id_value))
            
            connection.commit()
    except pymysql.MySQLError as e:
        print(f"Error: {e}")
    finally:
        if connection:
            connection.close()
            
            
increment_road(1,1,True)

In [9]:
import pymysql

# Database connection details
RDS_HOST = "database-2.cnqamusmkwon.eu-north-1.rds.amazonaws.com"
RDS_PORT = 3306  # MySQL default port
DB_USER = "root"
DB_PASSWORD = "mkmk162345"
DB_NAME = "traffic"

# Function to connect to the database
def get_connection():
    return pymysql.connect(
        host=RDS_HOST,
        port=RDS_PORT,
        user=DB_USER,
        password=DB_PASSWORD,
        database=DB_NAME,
        cursorclass=pymysql.cursors.DictCursor
    )

# Function to display the current road_current value
def display_road_current(id_value):
    try:
        connection = get_connection()
        with connection.cursor() as cursor:
            # Step 1: Retrieve the current road_current value for the given ID
            select_sql = "SELECT * FROM traffic_counter_road WHERE id = %s"
            cursor.execute(select_sql, (id_value,))
            result = cursor.fetchone()
        
        # Step 2: Display the road_current value
        if result:
            print(f"Current road_current for ID {id_value}: {result['road_current']}, {result['road_in']}, {result['road_out']}")
        else:
            print(f"No data found for ID {id_value}")
    
    except pymysql.MySQLError as e:
        print(f"Error: {e}")
    
    finally:
        if connection:
            connection.close()

# Example usage:
display_road_current(1)  # Replace 'some_id_value' with the actual ID you want to check


Current road_current for ID 1: 195, 223, 28


In [39]:
import pymysql

# Database connection details
RDS_HOST = "database-2.cnqamusmkwon.eu-north-1.rds.amazonaws.com"
RDS_PORT = 3306  # MySQL default port
DB_USER = "root"
DB_PASSWORD = "mkmk162345"
DB_NAME = "traffic"

# Function to connect to the database
def get_connection():
    return pymysql.connect(
        host=RDS_HOST,
        port=RDS_PORT,
        user=DB_USER,
        password=DB_PASSWORD,
        database=DB_NAME,
        cursorclass=pymysql.cursors.DictCursor
    )

# Function to insert data into the table
def insert_data(id_value, road_in_value, road_out_value):
    try:
        connection = get_connection()
        with connection.cursor() as cursor:
            # Step 1: Calculate road_current
            road_current_value = road_in_value - road_out_value
            
            # Step 2: Insert data into the table
            insert_sql = """
            INSERT INTO traffic_counter_road (id, road_in, road_out, road_current)
            VALUES (%s, %s, %s, %s)
            """
            cursor.execute(insert_sql, (id_value, road_in_value, road_out_value, road_current_value))
        
        # Commit the changes
        connection.commit()
        print(f"Data inserted successfully! ID: {id_value}, Road In: {road_in_value}, Road Out: {road_out_value}, Road Current: {road_current_value}")
    
    except pymysql.MySQLError as e:
        print(f"Error: {e}")
    
    finally:
        if connection:
            connection.close()

# Example usage:
insert_data(1, 0, 0)  # Replace 'new_id_value' with your ID, and 100, 50 with road_in and road_out values


Error: (1062, "Duplicate entry '1' for key 'traffic_counter_road.PRIMARY'")


## Creating Database

In [3]:
import pymysql

RDS_HOST = "database-2.cnqamusmkwon.eu-north-1.rds.amazonaws.com"
RDS_PORT = 3306  # MySQL default port
DB_USER = "root"
DB_PASSWORD = "mkmk162345"
DB_NAME = "traffic"

# Connect to the RDS MySQL instance
try:
    connection = pymysql.connect(
        host=RDS_HOST,
        port=RDS_PORT,
        user=DB_USER,
        password=DB_PASSWORD,
        database=DB_NAME,
        cursorclass=pymysql.cursors.DictCursor
    )
    print("Connected to the database!")

    # Create a table
    with connection.cursor() as cursor:
        create_table_query = """
        CREATE TABLE license_plates (
            license_plate VARCHAR(25)
        );
        """
        cursor.execute(create_table_query)
        print("Table 'traffic_data' created successfully!")

    # Commit the changes
    connection.commit()

except pymysql.MySQLError as e:
    print(f"Error: {e}")

finally:
    if connection:
        connection.close()
        print("Connection closed.")


Connected to the database!
Table 'traffic_data' created successfully!
Connection closed.


## Inserting to Database

In [ ]:
import pymysql

RDS_HOST = "database-2.cnqamusmkwon.eu-north-1.rds.amazonaws.com"
RDS_PORT = 3306  # MySQL default port
DB_USER = "root"
DB_PASSWORD = "mkmk162345"
DB_NAME = "traffic"

# Connect to the RDS MySQL instance
try:
    connection = pymysql.connect(
        host=RDS_HOST,
        port=RDS_PORT,
        user=DB_USER,
        password=DB_PASSWORD,
        database=DB_NAME,
        cursorclass=pymysql.cursors.DictCursor
    )
    print("Connected to the database!")

    with connection.cursor() as cursor:
        insert_query = """
        INSERT INTO traffic_data 
        (road_name, distance_km, duration_in_traffic_min, speed_kmh, delay_min, traffic_condition, timestamp, start_point, end_point) 
        VALUES (%s, %s, %s, %s, %s, %s, %s, POINT(%s, %s), POINT(%s, %s))
        """
        
        # Insert each row of data
        for row in traffic_data:
            cursor.execute(insert_query, (
                row[0],  # road_name
                row[1],  # distance_km
                row[2],  # duration_in_traffic_min
                row[3],  # speed_kmh
                row[4],  # delay_min
                row[5],  # traffic_condition
                row[6],  # timestamp
                row[7][0], row[7][1],  # start_point (latitude, longitude)
                row[8][0], row[8][1]   # end_point (latitude, longitude)
            ))

    # Commit the transaction
    connection.commit()
    print("Data inserted successfully!")

except pymysql.MySQLError as e:
    print(f"Error: {e}")

finally:
    if connection:
        connection.close()
        print("Connection closed.")


## Retrieve the whole data from s3 bucket

In [14]:
# import boto3
# import pandas as pd
# from io import StringIO

# # Initialize the S3 client
# s3_client = boto3.client('s3')

# # Your S3 bucket name
# bucket_name = 'savingbuckett5'

# # Prefix to filter the files (traffic_data_*)
# prefix = 'traffic_data_'

# # Get a list of files with the specified prefix from the S3 bucket
# response = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=prefix)

# # List to hold dataframes from all files
# dfs = []

# # Loop through each file in the S3 bucket that matches the prefix
# for obj in response.get('Contents', []):
#     file_key = obj['Key']  # The key is the file name in S3
    
#     # Only process files that start with 'traffic_data_'
#     if file_key.startswith(prefix):
#         # Read the file from S3
#         csv_obj = s3_client.get_object(Bucket=bucket_name, Key=file_key)
        
#         # Read the content of the file
#         csv_string = csv_obj['Body'].read().decode('utf-8')
        
#         # Convert the CSV string to a pandas DataFrame
#         df = pd.read_csv(StringIO(csv_string))
        
#         # Append the DataFrame to the list
#         dfs.append(df)

# # Concatenate all dataframes into one
# final_df = pd.concat(dfs, ignore_index=True)

## Retrieving Dataset from MySQL

In [6]:
import pymysql
import pandas as pd

RDS_HOST = "database-2.cnqamusmkwon.eu-north-1.rds.amazonaws.com"
RDS_PORT = 3306  # MySQL default port
DB_USER = "root"
DB_PASSWORD = "mkmk162345"
DB_NAME = "traffic"

# SQL Query to retrieve data
query = "SELECT * FROM traffic_counter_road"  # Replace with your table name

# Connect to the RDS MySQL instance
try:
    connection = pymysql.connect(
        host=RDS_HOST,
        port=RDS_PORT,
        user=DB_USER,
        password=DB_PASSWORD,
        database=DB_NAME,
        cursorclass=pymysql.cursors.DictCursor  # Fetches the result as a dictionary
    )
    print("Connected to the database!")

    # Execute the query and fetch the data
    with connection.cursor() as cursor:
        cursor.execute(query)
        results = cursor.fetchall()  # Fetch all rows from the table
        
        # Convert the query results into a Pandas DataFrame
        df = pd.DataFrame(results)
        print("Data retrieved and loaded into a DataFrame!")

    # Show the first 5 rows of the DataFrame
    print(df.head())

except pymysql.MySQLError as e:
    print(f"Error: {e}")

finally:
    if connection:
        connection.close()
        print("Connection closed.")


Connected to the database!
Data retrieved and loaded into a DataFrame!
   id  road_in  road_out  road_current  truck_cost  motorbike_cost  car_cost  \
0   1      233        33           200         NaN             NaN       NaN   
1   2        5         2             3        16.0             8.0       5.0   

   vehicle_count  
0            NaN  
1            3.0  
Connection closed.


In [3]:
# import pymysql
# import pandas as pd

# RDS_HOST = "database-2.cnqamusmkwon.eu-north-1.rds.amazonaws.com"
# RDS_PORT = 3306  # MySQL default port
# DB_USER = "root"
# DB_PASSWORD = "mkmk162345"
# DB_NAME = "traffic"

# # SQL Query to retrieve data
# query = """ALTER TABLE traffic_counter_road
# ADD COLUMN truck_cost INT,
# ADD COLUMN motorbike_cost INT,
# ADD COLUMN car_cost INT,
# ADD COLUMN vehicle_count INT;"""  # Replace with your table name

# # Connect to the RDS MySQL instance
# try:
#     connection = pymysql.connect(
#         host=RDS_HOST,
#         port=RDS_PORT,
#         user=DB_USER,
#         password=DB_PASSWORD,
#         database=DB_NAME,
#         cursorclass=pymysql.cursors.DictCursor  # Fetches the result as a dictionary
#     )
#     print("Connected to the database!")

#     # Execute the query and fetch the data
#     with connection.cursor() as cursor:
#         cursor.execute(query)
#         # results = cursor.fetchall()  # Fetch all rows from the table
#         print("Columns added to the table!")
#         # Convert the query results into a Pandas DataFrame
#         # df = pd.DataFrame(results)
#         # print("Data retrieved and loaded into a DataFrame!")

#     # Show the first 5 rows of the DataFrame
#     print(df.head())

# except pymysql.MySQLError as e:
#     print(f"Error: {e}")

# finally:
#     if connection:
#         connection.close()
#         print("Connection closed.")


Connected to the database!
Error: (1060, "Duplicate column name 'truck_cost'")
Connection closed.


In [11]:
# import pymysql
# import pandas as pd

# RDS_HOST = "database-2.cnqamusmkwon.eu-north-1.rds.amazonaws.com"
# RDS_PORT = 3306  # MySQL default port
# DB_USER = "root"
# DB_PASSWORD = "mkmk162345"
# DB_NAME = "traffic"

# # SQL Query to retrieve data
# query = """DELETE FROM traffic_counter_road;"""  # Replace with your table name

# # Connect to the RDS MySQL instance
# try:
#     connection = pymysql.connect(
#         host=RDS_HOST,
#         port=RDS_PORT,
#         user=DB_USER,
#         password=DB_PASSWORD,
#         database=DB_NAME,
#         cursorclass=pymysql.cursors.DictCursor  # Fetches the result as a dictionary
#     )
#     print("Connected to the database!")

#     # Execute the query and fetch the data
#     with connection.cursor() as cursor:
#         cursor.execute(query)
#         # results = cursor.fetchall()  # Fetch all rows from the table
#         print("Columns added to the table!")
#         # Convert the query results into a Pandas DataFrame
#         # df = pd.DataFrame(results)
#         # print("Data retrieved and loaded into a DataFrame!")

#     # Show the first 5 rows of the DataFrame
#     print(df.head())

# except pymysql.MySQLError as e:
#     print(f"Error: {e}")

# finally:
#     if connection:
#         connection.close()
#         print("Connection closed.")


Connected to the database!
Columns added to the table!
   id  road_in  road_out  road_current truck_cost motorbike_cost car_cost  \
0   1      233        33           200       None           None     None   

  vehicle_count  
0          None  
Connection closed.


In [5]:
import pymysql

RDS_HOST = "database-2.cnqamusmkwon.eu-north-1.rds.amazonaws.com"
RDS_PORT = 3306  # MySQL default port
DB_USER = "root"
DB_PASSWORD = "mkmk162345"
DB_NAME = "traffic"

# Connect to the RDS MySQL instance
try:
    connection = pymysql.connect(
        host=RDS_HOST,
        port=RDS_PORT,
        user=DB_USER,
        password=DB_PASSWORD,
        database=DB_NAME,
        cursorclass=pymysql.cursors.DictCursor
    )
    print("Connected to the database!")

    with connection.cursor() as cursor:
        insert_query = """
        INSERT INTO traffic_counter_road 
        (id, road_in, road_out,road_current, truck_cost, motorbike_cost, car_cost, vehicle_count) 
        VALUES (%s, %s,%s, %s, %s, %s, %s, %s)
        """
        
        # Insert each row of data
        cursor.execute(insert_query, (
            2,  # road_name
            5,  # distance_km
            2,  # duration_in_traffic_min
            3,  # speed_kmh
            16,  # delay_min
            8,  # traffic_condition
            5,  # timestamp
            3
        ))

    # Commit the transaction
    connection.commit()
    print("Data inserted successfully!")

except pymysql.MySQLError as e:
    print(f"Error: {e}")

finally:
    if connection:
        connection.close()
        print("Connection closed.")


Connected to the database!
Data inserted successfully!
Connection closed.


In [7]:
# Define the new car cost and the ID of the row you want to update
new_car_cost = 9  # Replace with the actual new car cost
row_id = 2  # Replace with the ID of the row you want to update

try:
    connection = pymysql.connect(
        host=RDS_HOST,
        port=RDS_PORT,
        user=DB_USER,
        password=DB_PASSWORD,
        database=DB_NAME,
        cursorclass=pymysql.cursors.DictCursor
    )
    print("Connected to the database!")

    with connection.cursor() as cursor:
        update_query = """
        UPDATE traffic_counter_road
        SET car_cost = %s
        WHERE id = %s
        """
        
        # Execute the update query
        cursor.execute(update_query, (new_car_cost, row_id))

    # Commit the transaction
    connection.commit()
    print("Car cost updated successfully!")

except pymysql.MySQLError as e:
    print(f"Error: {e}")

finally:
    if connection:
        connection.close()
        print("Connection closed.")


Connected to the database!
Car cost updated successfully!
Connection closed.


## Download Data from S3 Bucket

In [6]:
import boto3
import os

def download_s3_csv_files(bucket_name, local_dir):
    # Initialize a session using Amazon S3
    s3 = boto3.resource('s3')

    # Select your S3 bucket
    bucket = s3.Bucket(bucket_name)

    # Iterate over all objects in the bucket
    for obj in bucket.objects.filter(Prefix=''):
        # Only download files that have .csv extension and are in the main bucket (no folder structure)
        if obj.key.endswith('.csv') and '/' not in obj.key:
            local_file_path = os.path.join(local_dir, obj.key)

            # Ensure the local directory exists
            if not os.path.exists(local_file_path):
                os.makedirs(local_dir, exist_ok=True)

            # Download the file
            print(f'Downloading {obj.key} to {local_file_path}')
            bucket.download_file(obj.key, local_file_path)

# Example usage:
bucket_name = 'savingbuckett5'
local_directory = './downloaded_files'

download_s3_csv_files(bucket_name, local_directory)
